In [44]:
%load_ext autoreload

In [45]:
%autoreload 2

In [1]:
from src.parsers import mimic, hirid
from src.modeling import discovery, plots, querier
from src.utils import constants

In [2]:
import sys
# import win32com.client
import os
import pandas as pd

def setup_io_config(root_path):
    """
    Input - Output config. Add dataset paths
    :root_path -> Repo path which contains 'data' and 'res' folders
    """

    # MIMIC
    is_shortcut = True if "data.lnk" in os.listdir(root_path) else False 
    
    if (is_shortcut):
        path_shortcut =  os.path.join(root_path, "data.lnk")
        shell = win32com.client.Dispatch("WScript.Shell")
        mimic_data = shell.CreateShortCut(path_shortcut).Targetpath
    else:
        mimic_data = os.path.join(f"{root_path}", "data") 
    mimic_path = os.path.join(f"{root_path}", "results")

    # HIRID
    hirid_data = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1/raw_stage/'
    hirid_path = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1'
    
    return mimic_data, mimic_path, hirid_data, hirid_path

def setup_stratification_config():
    gender="MF"
    age_b=40
    age_a=80 
    ethnicity="WHITE" 
    lab_mapping= constants.LAB_MAPPING
    before_windows = [(0,6), (6,12)]
    after_windows = [(0,4), (4,8), (8,12)]
    return gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows

In [3]:
# IO Config
# root_path ="C:\\Users\\danco\\My Drive\\Master\\Datasets\\MIMIC iii"
root_path = "/Users/pavan/Library/CloudStorage/GoogleDrive-f20190038@hyderabad.bits-pilani.ac.in/My Drive/TAU/Code/DrugLab"
data, res, raw_path, res_path = setup_io_config(root_path=root_path)

# Stratification Config
gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows = setup_stratification_config()

In [9]:
import traceback
try:
    mimic_parser = mimic.MIMICParser(data=data, res=res, gender=gender, age_b=age_b, age_a=age_a, ethnicity=ethnicity, load="AUTOMATIC_MAPPING_MIMIC")
    m_med1, m_med2, m_labs = mimic_parser.parse(use_pairs=False, load_from_raw=False, load_raw_chartevents=False)
except:
    traceback.print_exc()

Loading med data...
Loaded med data.
Load 1st and 2nd medication data...
Loaded 1st and 2nd medication data.
Load Lab data...


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:225: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  labs = pd.read_csv(os.path.join(self.data, constants.MIMIC_III_PREPROCESSED_PATH, constants.MIMIC_III_PREPROCESSED_LABDATA))


Loaded Lab data.


In [4]:
mimic_data_querier = querier.DatasetQuerier(
    data = data,
    res = res,
    t_labs=m_labs, 
    t_med1=m_med1, 
    t_med2=m_med2,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)

NameError: name 'm_labs' is not defined

In [11]:
m_final_lab_med_data = mimic_data_querier.generate_med_lab_data(before_windows, after_windows)

/Users/pavan/Desktop/TAU/DrugLab/src/modeling/querier.py:128: FutureWarning: Passing 'suffixes' which cause duplicate columns {'after_trends_(6, 12)_x', 'after_trends_(0, 4)_x', 'after_mean_(4, 8)_x', 'after_trends_(4, 8)_x', 'before_mean_(6, 12)_x', 'before_trends_(0, 6)_x', 'after_mean_(8, 12)_x', 'after_mean_(0, 4)_x', 'after_trends_(8, 12)_x', 'before_trends_(6, 12)_x', 'before_mean_(0, 6)_x'} in the result is deprecated and will raise a MergeError in a future version.
  col_vals[i] = pd.merge(col_vals[i-1], col_vals[i], how="outer", on=list(t_med1.columns)+["LAB_ITEMID"])
/Users/pavan/Desktop/TAU/DrugLab/src/modeling/querier.py:128: FutureWarning: Passing 'suffixes' which cause duplicate columns {'after_trends_(6, 12)_x', 'after_trends_(0, 4)_x', 'after_mean_(4, 8)_x', 'after_trends_(4, 8)_x', 'before_mean_(6, 12)_x', 'before_trends_(0, 6)_x', 'after_mean_(8, 12)_x', 'after_mean_(0, 4)_x', 'after_trends_(8, 12)_x', 'before_trends_(6, 12)_x', 'before_mean_(0, 6)_x'} in the result i

In [12]:
# med_lab_pair_data = m_final_lab_med_data[0]
med_lab_pair_data = pd.read_csv(os.path.join(data, ""))

In [13]:
analyzer = discovery.ClinicalDiscoveryAnalysis(med_lab_pair_data)
lab_name = 'Glucose'
med_name = 'Insulin - Regular'

In [ ]:
res_comp = analyzer.analyze(before_windows, after_windows)
res_comp = res_comp.dropna(subset=["TTest Paired"])

In [17]:
res_dict = res_comp.to_dict()

In [ ]:
res_comp.to_csv("temp.csv")

In [ ]:
res_comp_df_corrected = pd.DataFrame(res_dict[0].values())

In [ ]:
res_comp_df_corrected

In [ ]:
sig_pairs = analyzer.generate_significant(res_comp)

In [9]:
res_sig = pd.read_csv("temp_fdr_sig_pairs.csv").drop(columns=["Unnamed: 0"])

In [10]:
res_sig

,Lab Name,Med Name,Before Window (in Hours),After Window (in Hours),Mannwhitneyu Test,TTest Independent,TTest Paired,BonferroniPvals,FDR Benjamini Corrected,No. of Patients
0,Hemoglobin,Albumin 25%,"(0, 6)","(0, 4)",2.165030e-01,2.088679e-01,6.524663e-05,3.170986e-01,2.157133e-03,57
1,Hematocrit,Albumin 5%,"(0, 6)","(0, 4)",4.629792e-08,2.784454e-08,2.127214e-23,1.033826e-19,6.461411e-21,382
2,Hemoglobin,Albumin 5%,"(0, 6)","(0, 4)",3.244294e-03,2.738292e-03,3.215162e-08,1.562569e-04,2.441514e-06,173
3,Lactic acid,Albumin 5%,"(0, 6)","(0, 4)",2.044467e-02,1.553298e-01,1.039934e-05,5.054078e-02,4.396323e-04,177
4,Platelets,Albumin 5%,"(0, 6)","(0, 4)",3.664783e-01,4.097745e-01,6.902470e-05,3.354601e-01,2.251410e-03,130
...,...,...,...,...,...,...,...,...,...,...
175,Hematocrit,Vancomycin,"(0, 6)","(0, 4)",1.882328e-02,9.458800e-03,1.466061e-10,7.125055e-07,1.583345e-08,873
176,Hemoglobin,Vancomycin,"(0, 6)","(0, 4)",2.245281e-01,1.475547e-01,2.766560e-04,1.000000e+00,7.771954e-03,499
177,Lactic acid,Vancomycin,"(0, 6)","(0, 4)",3.765819e-02,2.654006e-01,9.722280e-05,4.725028e-01,3.028864e-03,396
178,Magnesium,Vancomycin,"(0, 6)","(0, 4)",1.506045e-02,4.091403e-02,3.326912e-04,1.000000e+00,9.032844e-03,440


In [91]:
res_dis_1 = analyzer.statistical_tests(med_name, lab_name, before_windows, after_windows)

In [92]:
pd.DataFrame(res_dis_1)

,Lab Name,Med Name,Before Window (in Hours),After Window (in Hours),Mannwhitneyu Test,TTest Independent,TTest Paired
0,Glucose,Insulin - Regular,"(0, 12)","(0, 12)",1.295140e-43,1.708010e-19,6.662180e-50


In [ ]:
pd.DataFrame(res_dis_1)

,Lab Name,Med Name,Before Window (in Hours),After Window (in Hours),Mannwhitneyu Test,TTest Independent,TTest Paired
0,Glucose,Insulin - Regular,"before_abs_(0, 6)_sp","after_abs_(0, 4)_sp",2.189386e-14,1.485992e-07,1.923674e-19
1,Glucose,Insulin - Regular,"before_abs_(6, 12)_sp","after_abs_(0, 4)_sp",4.326885e-13,7.642086e-02,1.329148e-02
2,Glucose,Insulin - Regular,"before_abs_(0, 6)_sp","after_abs_(4, 8)_sp",7.587147e-14,1.444190e-11,6.954230e-20
3,Glucose,Insulin - Regular,"before_abs_(6, 12)_sp","after_abs_(4, 8)_sp",5.304810e-01,1.426278e-01,7.623842e-02
4,Glucose,Insulin - Regular,"before_abs_(0, 6)_sp","after_abs_(8, 12)_sp",8.997221e-09,6.818553e-07,1.101460e-10
5,Glucose,Insulin - Regular,"before_abs_(6, 12)_sp","after_abs_(8, 12)_sp",8.560342e-02,4.862437e-02,6.028435e-03
